# Energy Ministry Cybersecurity Prioritization World Map

This notebook creates an interactive world map visualization showing how much each country's energy ministry prioritizes cybersecurity, based on the assessment data from `energy_ministry_cyber_full_assessment.csv`.

## Color Coding:
- **Dark Red**: High cybersecurity prioritization
- **Orange**: Medium cybersecurity prioritization  
- **Light Yellow**: Low cybersecurity prioritization
- **Light Gray**: No cybersecurity prioritization
- **White**: No data available

In [ ]:
# Import required libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
import json

# Initialize plotly for Jupyter
init_notebook_mode(connected=True)

print("Libraries imported successfully!")

In [ ]:
# Load the energy ministry cybersecurity assessment data
df = pd.read_csv('output/energy_ministry_cyber_full_assessment.csv')

print(f"Loaded data for {len(df)} countries")
print("\nCybersecurity prioritization distribution:")
print(df['relevance_level'].value_counts())

# Display sample data
print("\nSample data:")
print(df[['country', 'relevance_level', 'confidence']].head(10))

In [ ]:
# Create color mapping for cybersecurity prioritization levels
color_map = {
    'high': '#8B0000',      # Dark Red
    'medium': '#FF8C00',    # Orange  
    'low': '#FFD700',       # Gold/Light Yellow
    'none': '#D3D3D3'       # Light Gray
}

# Add color column to dataframe
df['color'] = df['relevance_level'].map(color_map)

# Add numeric values for plotly choropleth
relevance_to_numeric = {
    'none': 0,
    'low': 1, 
    'medium': 2,
    'high': 3
}
df['relevance_numeric'] = df['relevance_level'].map(relevance_to_numeric)

# Create hover text with detailed information
df['hover_text'] = df.apply(lambda row: 
    f"<b>{row['country']}</b><br>" +
    f"Cybersecurity Priority: {row['relevance_level'].title()}<br>" +
    f"Assessment Confidence: {row['confidence'].title()}<br>" +
    f"Energy Ministry Involvement: {row['relevance_level'].title()}", axis=1)

print("Data preparation completed!")
print(f"Countries with high prioritization: {len(df[df['relevance_level'] == 'high'])}")
print(f"Countries with medium prioritization: {len(df[df['relevance_level'] == 'medium'])}")
print(f"Countries with low prioritization: {len(df[df['relevance_level'] == 'low'])}")
print(f"Countries with no prioritization: {len(df[df['relevance_level'] == 'none'])}")

In [ ]:
# Handle country name mapping for plotly compatibility
# Some countries in our data might have different names than expected by plotly
country_name_mapping = {
    'United States of America': 'United States',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Russian Federation': 'Russia',
    'Iran, Islamic Republic of': 'Iran',
    'Venezuela, Bolivarian Republic of': 'Venezuela',
    'Bolivia, Plurinational State of': 'Bolivia',
    'Tanzania, United Republic of': 'Tanzania',
    'Syria, Arab Republic': 'Syria',
    'Moldova, Republic of': 'Moldova',
    'Korea, Democratic People\'s Republic of': 'North Korea',
    'Korea, Republic of': 'South Korea',
    'Congo, Democratic Republic of the': 'Democratic Republic of the Congo',
    'Congo': 'Republic of the Congo',
    'Lao People\'s Democratic Republic': 'Laos',
    'Macedonia, the former Yugoslav Republic of': 'North Macedonia',
    'Micronesia, Federated States of': 'Micronesia'
}

# Apply country name mapping
df['country_mapped'] = df['country'].replace(country_name_mapping)

print("Country name mapping applied!")
print(f"Sample mapped countries: {df['country_mapped'].head().tolist()}")

In [ ]:
# Create the world map using plotly
fig = px.choropleth(
    df,
    locations='country_mapped',
    locationmode='country names',
    color='relevance_numeric',
    hover_data=['country', 'relevance_level', 'confidence'],
    color_continuous_scale=[
        [0, '#D3D3D3'],    # None - Light Gray
        [0.33, '#FFD700'], # Low - Gold
        [0.66, '#FF8C00'], # Medium - Orange
        [1, '#8B0000']     # High - Dark Red
    ],
    title="<b>Energy Ministry Cybersecurity Prioritization by Country</b><br><sub>Based on assessment of 193 countries' energy ministries</sub>",
    labels={'relevance_numeric': 'Cybersecurity Priority Level',
            'country_mapped': 'Country'}
)

# Customize the map layout
fig.update_layout(
    title_x=0.5,  # Center the title
    title_font_size=20,
    width=1200,
    height=700,
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    )
)

# Update colorbar
fig.update_coloraxes(
    colorbar=dict(
        title="Cybersecurity Priority",
        tickvals=[0, 1, 2, 3],
        ticktext=["None", "Low", "Medium", "High"],
        len=0.7
    )
)

# Update hover template for better information display
fig.update_traces(
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Cybersecurity Priority: %{customdata[1]}<br>" +
                  "Assessment Confidence: %{customdata[2]}<br>" +
                  "<extra></extra>",
    customdata=df[['country', 'relevance_level', 'confidence']].values
)

# Display the map
fig.show()

print("Interactive world map created successfully!")

In [ ]:
# Display summary statistics
print("=" * 60)
print("ENERGY MINISTRY CYBERSECURITY ASSESSMENT SUMMARY")
print("=" * 60)

total_countries = len(df)
high_count = len(df[df['relevance_level'] == 'high'])
medium_count = len(df[df['relevance_level'] == 'medium'])
low_count = len(df[df['relevance_level'] == 'low'])
none_count = len(df[df['relevance_level'] == 'none'])

print(f"Total countries assessed: {total_countries}")
print(f"\nCybersecurity Prioritization Breakdown:")
print(f"  🔴 High Priority:   {high_count:3d} countries ({high_count/total_countries*100:.1f}%)")
print(f"  🟠 Medium Priority: {medium_count:3d} countries ({medium_count/total_countries*100:.1f}%)")
print(f"  🟡 Low Priority:    {low_count:3d} countries ({low_count/total_countries*100:.1f}%)")
print(f"  ⚪ No Priority:     {none_count:3d} countries ({none_count/total_countries*100:.1f}%)")

print(f"\nCountries with HIGH cybersecurity prioritization:")
high_countries = df[df['relevance_level'] == 'high']['country'].tolist()
for i, country in enumerate(sorted(high_countries)):
    if i % 3 == 0:
        print()
    print(f"{country:25}", end=" ")

print(f"\n\nCountries with NO cybersecurity prioritization:")
none_countries = df[df['relevance_level'] == 'none']['country'].tolist()
for country in sorted(none_countries):
    print(f"  • {country}")

print("\n" + "=" * 60)

In [ ]:
# Save the interactive map as an HTML file
output_filename = "energy_ministry_cybersecurity_world_map.html"
fig.write_html(output_filename)
print(f"Interactive map saved as '{output_filename}'")

# Also save a static image version
try:
    fig.write_image("energy_ministry_cybersecurity_world_map.png", width=1200, height=700)
    print("Static PNG image saved as 'energy_ministry_cybersecurity_world_map.png'")
except Exception as e:
    print(f"Note: Could not save PNG image: {e}")
    print("This is normal if kaleido is not installed. The HTML version contains the full interactive map.")

## Key Findings

This world map visualization reveals the global landscape of energy ministry involvement in cybersecurity:

### Geographic Patterns:
- **High Priority Countries**: Typically developed nations with significant energy infrastructure and cybersecurity awareness
- **Medium Priority Countries**: Mix of developing and developed nations with growing cybersecurity programs
- **Low Priority Countries**: Often developing nations where cybersecurity may not yet be a primary focus for energy ministries
- **No Priority Countries**: Smaller nations or those where energy and cybersecurity responsibilities are completely separate

### Insights:
1. **Regional Variations**: Clear patterns emerge by geographic region and economic development level
2. **Infrastructure Correlation**: Countries with critical energy infrastructure tend to have higher cybersecurity prioritization
3. **Policy Development**: The distribution suggests varying levels of national cybersecurity policy maturity

### Data Source:
Based on automated assessment of 193 countries' energy ministries using the AutoRA research framework, analyzing their role as cybersecurity stakeholders in their respective national contexts.